In [3]:
import numpy as np
from collections import defaultdict

from collections import namedtuple
import random
from random import choice

from abc import ABC, abstractmethod
from collections import defaultdict
import math

### (a) Implementing MCTS algorithm for Tic Tac Toe

In [191]:
class MCTS:

    def __init__(self, C=1):
        self.Q = defaultdict(int)
        self.N = defaultdict(int)
        self.children = {}
        self.C = C

    def choose(self, node):

        if node not in self.children:
            return node.get_rand_child()

        def avg_rew(n):
            if self.N[n] == 0:
                return float("-inf")
            return self.Q[n]/self.N[n]

        return max(self.children[node], key=avg_rew)

    def playout(self, node):
        path = self.Select(node)
        leaf = path[-1]
        self.Expand(leaf)
        reward = self.get_reward(leaf)
        self.back_prop(path, reward)

    def Select(self, node):
        path = []
        while True:
            path.append(node)
            if node not in self.children or not self.children[node]:
                return path
            unexplored = self.children[node] - self.children.keys()
            if unexplored:
                n = unexplored.pop()
                path.append(n)
                return path
            node = self.UCB_child(node)

    def get_reward(self, node):
        invert_reward = True
        while True:
            if node.is_terminal():
                reward = node.reward()
                return 1 - reward if invert_reward else reward
            node = node.get_rand_child()
            invert_reward = not invert_reward
    
    def Expand(self, node):
        if node in self.children:
            return
        self.children[node] = node.get_children()

    def back_prop(self, path, reward):
        for node in reversed(path):
            self.N[node] += 1
            self.Q[node] += reward
            reward = 1 - reward

    def UCB_child(self, node):

        assert all(n in self.children for n in self.children[node])
        node_logN = math.log(self.N[node])

        def ucb(n):
            return self.Q[n]/self.N[n] + self.C*math.sqrt(node_logN/self.N[n])

        return max(self.children[node], key=ucb)

In [192]:
class Node(ABC):
    
    @abstractmethod
    def get_children(self):
        return set()

    @abstractmethod
    def get_rand_child(self):
        return None

    @abstractmethod
    def is_terminal(self):
        return True

    @abstractmethod
    def reward(self):
        return 0

    @abstractmethod
    def __hash__(self):
        return 1234567890

    @abstractmethod
    def __eq__(node1, node2):
        return True

In [194]:
TTT = namedtuple("TicTacToeBoard", "tup turn winner terminal")

In [197]:
class TicTacToeBoard(TTT, Node):

    def get_children(board):
        if board.terminal:
            return set()
        return {board.make_move(i) for i, value in enumerate(board.tup) if value is None}

    def get_rand_child(board):
        if board.terminal:
            return None 
        empty_spots = [i for i, value in enumerate(board.tup) if value is None]
        return board.make_move(choice(empty_spots))

    def reward(board):
        if board.turn is (not board.winner):
            return 0
        if board.winner is None:
            return 0.5

    def is_terminal(board):
        return board.terminal

    def make_move(board, index):
        tup = board.tup[:index] + (board.turn,) + board.tup[index + 1 :]
        turn = not board.turn
        winner = find_winner(tup)
        is_terminal = (winner is not None) or not any(v is None for v in tup)
        return TicTacToeBoard(tup, turn, winner, is_terminal)

    def print_board(board):
        to_char = lambda v: ("X" if v is True else ("O" if v is False else " "))
        rows = [[to_char(board.tup[3*row+col]) for col in range(3)] for row in range(3)]
        print("\n  1 2 3\n" + "\n".join(str(i + 1) + " " + " ".join(row) for i, row in enumerate(rows)) + "\n")

In [198]:
def winning_states():
    return ((0, 1, 2), (3, 4, 5), (6, 7, 8), (0, 3, 6), (1, 4, 7), (2, 5, 8), (0, 4, 8), (2, 4, 6))

In [16]:
def find_winner(tup):
    "Returns None if no winner, True if X wins, False if O wins"
    for i1, i2, i3 in winning_states():
        v1, v2, v3 = tup[i1], tup[i2], tup[i3]
        if False is v1 is v2 is v3:
            return False
        if True is v1 is v2 is v3:
            return True
    return None

Making the Play Function to play games with MCTS agent against different agents

In [154]:
def play_game(K, agent, BOARD, first=0):

    mcts = MCTS()
    board = BOARD()
    print("NEW GAME")
    board.print_board()

    if(first!=0):
        print("MCTS Agent Turn:")
        for _ in range(K):
            mcts.playout(board)
        board = mcts.choose(board)
        board.print_board()
        if board.terminal:
            s = ""
            if((board.winner == True and first==0) or (board.winner == False and first==1)):
                s="LOSS"
            elif((board.winner == False and first==0) or (board.winner == True and first==1)):
                s = "WIN"
            else:
                s = "DRAW"
            print(s)
            return s
            
        
    while True:
        row, col = agent(board)
        index = 3*(row) + (col)
        
        board = board.make_move(index)
        print("Opponent Turn:")
        board.print_board()
        if board.terminal:
            break

        for _ in range(K):
            mcts.playout(board)
        board = mcts.choose(board)
        print("MCTS Agent Turn:")
        board.print_board()
        if board.terminal:
            break
    
    s = ""
    if((board.winner == True and first==0) or (board.winner == False and first==1)):
        s = "LOSS"
    elif((board.winner == False and first==0) or (board.winner == True and first==1)):
        s = "WIN"
    else:
        s = "DRAW"

    print(s)
    return s

In [65]:
def rand_agent(board):

    valid_moves = []

    for i in range(len(board.tup)):
        if(board.tup[i]==None):
            valid_moves.append(i)
    
    action = choice(valid_moves)

    row = int(action/3)
    col = action%3
        
    return row, col

In [66]:
def safe_agent(board):

    valid_moves = []

    for i in range(len(board.tup)):
        if(board.tup[i]==None):
            valid_moves.append(i)

    for idx in valid_moves:
        new_tup = board.tup[:idx] + (board.turn,) + board.tup[idx+1:]
        for s1, s2, s3 in winning_states():
            if(new_tup[s1]==board.turn and new_tup[s2]==board.turn and new_tup[s3]==board.turn):
                row = int(idx/3)
                col = idx%3
                return row, col
        for s1, s2, s3 in winning_states():
            if((new_tup[s1]==board.turn and new_tup[s2]==board.turn and new_tup[s3]==(not board.turn)) or (new_tup[s1]==(not board.turn) and new_tup[s2]==board.turn and new_tup[s3]== board.turn) or (new_tup[s1]==board.turn and new_tup[s2]==(not board.turn) and new_tup[s3]== board.turn)):
                row = int(idx/3)
                col = idx%3
                return row, col

    action = random.choice(valid_moves)

    row = int(action/3)
    col = action%3
        
    return row, col

In [140]:
def NEW_BOARD():
    return TicTacToeBoard(tup=(None,) * 9, turn=True, winner=None, terminal=False)

In [170]:
K = 100 # number of simulations

In [155]:
_ = play_game(K, rand_agent, NEW_BOARD, 1) # Playing a game against random agent

NEW GAME

  1 2 3
1      
2      
3      

MCTS Agent Turn:

  1 2 3
1      
2   X  
3      

Opponent Turn:

  1 2 3
1     O
2   X  
3      

MCTS Agent Turn:

  1 2 3
1   X O
2   X  
3      

Opponent Turn:

  1 2 3
1 O X O
2   X  
3      

MCTS Agent Turn:

  1 2 3
1 O X O
2   X  
3   X  

WIN


### (b) 

(i) Case when MCTS agent is one move away from win

In [156]:
def BOARD1():
    return TicTacToeBoard(tup=(True, None, False, None, True, False, False, True, None), turn=True, winner=None, terminal=False)

In [157]:
_ = play_game(K, safe_agent, BOARD1, 1)

NEW GAME

  1 2 3
1 X   O
2   X O
3 O X  

MCTS Agent Turn:

  1 2 3
1 X X O
2   X O
3 O X  

WIN


The MCTS agent found the winning move

(ii) Case when MCTS agent is one move away from loss

In [162]:
def BOARD2():
    return TicTacToeBoard(tup=(False, False, None, True, None, None, None, True, None), turn=True, winner=None, terminal=False)

In [163]:
_ = play_game(K, safe_agent, BOARD2, 1)

NEW GAME

  1 2 3
1 O O  
2 X    
3   X  

MCTS Agent Turn:

  1 2 3
1 O O X
2 X    
3   X  

Opponent Turn:

  1 2 3
1 O O X
2 X O  
3   X  

MCTS Agent Turn:

  1 2 3
1 O O X
2 X O  
3   X X

Opponent Turn:

  1 2 3
1 O O X
2 X O O
3   X X

MCTS Agent Turn:

  1 2 3
1 O O X
2 X O O
3 X X X

WIN


The MCTS agent found the best move to avoid LOSS

(iii) Case when opponent has made the first move and occupied center square.

In [164]:
def BOARD3():
    return TicTacToeBoard(tup=(None, None, None, None, False, None, None, None, None), turn=True, winner=None, terminal=False)

In [165]:
_ = play_game(K, safe_agent, BOARD3, 1)

NEW GAME

  1 2 3
1      
2   O  
3      

MCTS Agent Turn:

  1 2 3
1      
2   O  
3 X    

Opponent Turn:

  1 2 3
1     O
2   O  
3 X    

MCTS Agent Turn:

  1 2 3
1 X   O
2   O  
3 X    

Opponent Turn:

  1 2 3
1 X O O
2   O  
3 X    

MCTS Agent Turn:

  1 2 3
1 X O O
2 X O  
3 X    

WIN


The MCTS agent played a corner square and managed to win the game

### (c)

(i) Recording wins, losses and draws with random agent

In [180]:
WIN = 0
DRAW = 0
LOSS = 0

for i in range(1000):
    print("Game ", i+1)
    s = play_game(K, rand_agent, NEW_BOARD, choice([0, 1]))
    if(s=="WIN"):
        WIN+=1
    elif(s=="LOSS"):
        LOSS+=1
    else:
        DRAW+=1

Streaming output truncated to the last 5000 lines.
3     X

MCTS Agent Turn:

  1 2 3
1   O X
2 O   X
3     X

WIN
Game  907
NEW GAME

  1 2 3
1      
2      
3      

Opponent Turn:

  1 2 3
1      
2      
3     X

MCTS Agent Turn:

  1 2 3
1      
2   O  
3     X

Opponent Turn:

  1 2 3
1      
2   O  
3   X X

MCTS Agent Turn:

  1 2 3
1      
2   O  
3 O X X

Opponent Turn:

  1 2 3
1     X
2   O  
3 O X X

MCTS Agent Turn:

  1 2 3
1     X
2   O O
3 O X X

Opponent Turn:

  1 2 3
1     X
2 X O O
3 O X X

MCTS Agent Turn:

  1 2 3
1   O X
2 X O O
3 O X X

Opponent Turn:

  1 2 3
1 X O X
2 X O O
3 O X X

DRAW
Game  908
NEW GAME

  1 2 3
1      
2      
3      

Opponent Turn:

  1 2 3
1   X  
2      
3      

MCTS Agent Turn:

  1 2 3
1   X  
2   O  
3      

Opponent Turn:

  1 2 3
1   X  
2   O  
3   X  

MCTS Agent Turn:

  1 2 3
1   X  
2   O O
3   X  

Opponent Turn:

  1 2 3
1 X X  
2   O O
3   X  

MCTS Agent Turn:

  1 2 3
1 X X O
2   O O
3   X  

Opponent Turn:

  1 2 3
1

In [183]:
print("WINS: ", WIN, "/ 1000")
print("LOSS: ", LOSS, "/ 1000")
print("DRAWS: ", DRAW, "/ 1000")

WINS:  967 / 1000
LOSS:  10 / 1000
DRAWS:  23 / 1000


(ii) Recording wins, losses and draws with safe agent

In [182]:
WIN = 0
DRAW = 0
LOSS = 0

for i in range(1000):
    print("Game ", i+1)
    s = play_game(K, safe_agent, NEW_BOARD, choice([0, 1]))
    if(s=="WIN"):
        WIN+=1
    elif(s=="LOSS"):
        LOSS+=1
    else:
        DRAW+=1

Streaming output truncated to the last 5000 lines.

  1 2 3
1 X    
2   O  
3 X   O

MCTS Agent Turn:

  1 2 3
1 X    
2 X O  
3 X   O

WIN
Game  902
NEW GAME

  1 2 3
1      
2      
3      

MCTS Agent Turn:

  1 2 3
1      
2      
3     X

Opponent Turn:

  1 2 3
1      
2   O  
3     X

MCTS Agent Turn:

  1 2 3
1      
2   O  
3 X   X

Opponent Turn:

  1 2 3
1 O    
2   O  
3 X   X

MCTS Agent Turn:

  1 2 3
1 O    
2   O  
3 X X X

WIN
Game  903
NEW GAME

  1 2 3
1      
2      
3      

MCTS Agent Turn:

  1 2 3
1      
2   X  
3      

Opponent Turn:

  1 2 3
1     O
2   X  
3      

MCTS Agent Turn:

  1 2 3
1     O
2   X  
3     X

Opponent Turn:

  1 2 3
1     O
2   X O
3     X

MCTS Agent Turn:

  1 2 3
1 X   O
2   X O
3     X

WIN
Game  904
NEW GAME

  1 2 3
1      
2      
3      

MCTS Agent Turn:

  1 2 3
1 X    
2      
3      

Opponent Turn:

  1 2 3
1 X    
2      
3 O    

MCTS Agent Turn:

  1 2 3
1 X   X
2      
3 O    

Opponent Turn:

  1 2 3
1 X   X
2 O    


In [181]:
print("WINS: ", WIN, "/ 1000")
print("LOSS: ", LOSS, "/ 1000")
print("DRAWS: ", DRAW, "/ 1000")

WINS:  914 / 1000
LOSS:  15 / 1000
DRAWS:  71 / 1000


The MCTS Agent played better than both the agents

### (d) Playing with MCTS Agent

In [184]:
def MCTS_AGENT_opp(board):
    mcts1 = MCTS()
    for _ in range(100):
        mcts1.playout(board)
    board1 = mcts1.choose(board)
    
    move = 0
    for i in range(len(board.tup)):
        if(board.tup[i]!=board1.tup[i]):
            move=i
    
    return int(move/3), move%3

In [187]:
WIN = 0
DRAW = 0
LOSS = 0

for i in range(1000):
    print("Game ", i+1)
    s = play_game(K, MCTS_AGENT_opp, NEW_BOARD, choice([0, 1]))
    if(s=="WIN"):
        WIN+=1
    elif(s=="LOSS"):
        LOSS+=1
    else:
        DRAW+=1

Streaming output truncated to the last 5000 lines.

  1 2 3
1 X    
2   O  
3 X    

MCTS Agent Turn:

  1 2 3
1 X    
2 O O  
3 X    

Opponent Turn:

  1 2 3
1 X    
2 O O X
3 X    

MCTS Agent Turn:

  1 2 3
1 X    
2 O O X
3 X O  

Opponent Turn:

  1 2 3
1 X X  
2 O O X
3 X O  

MCTS Agent Turn:

  1 2 3
1 X X O
2 O O X
3 X O  

Opponent Turn:

  1 2 3
1 X X O
2 O O X
3 X O X

DRAW
Game  923
NEW GAME

  1 2 3
1      
2      
3      

MCTS Agent Turn:

  1 2 3
1      
2      
3     X

Opponent Turn:

  1 2 3
1      
2      
3 O   X

MCTS Agent Turn:

  1 2 3
1     X
2      
3 O   X

Opponent Turn:

  1 2 3
1     X
2     O
3 O   X

MCTS Agent Turn:

  1 2 3
1 X   X
2     O
3 O   X

Opponent Turn:

  1 2 3
1 X   X
2   O O
3 O   X

MCTS Agent Turn:

  1 2 3
1 X X X
2   O O
3 O   X

WIN
Game  924
NEW GAME

  1 2 3
1      
2      
3      

MCTS Agent Turn:

  1 2 3
1      
2      
3   X  

Opponent Turn:

  1 2 3
1      
2      
3   X O

MCTS Agent Turn:

  1 2 3
1   X  
2      
3   X O

In [188]:
print("WINS: ", WIN, "/ 1000")
print("LOSS: ", LOSS, "/ 1000")
print("DRAWS: ", DRAW, "/ 1000")

WINS:  216 / 1000
LOSS:  204 / 1000
DRAWS:  580 / 1000


We can see that both the Agents performed almost equally well, since they were same.